## Setup

In [1]:
import requests
import pandas as pd
import os
import glob
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#load the district files into a single dataframe
dist_path = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/full_districts"
all_files = sorted(glob.glob(dist_path + "/*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

districts = pd.concat(li, axis=0, ignore_index=False)

In [3]:
state_path = "/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/clean_data/state_pvi"
all_files = sorted(glob.glob(state_path + "/*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

all_states = pd.concat(li, axis=0, ignore_index=False)

In [4]:
red = [2010,2014]
blu = [2006,2008,2018]
districts["wave"] = "Neutral"
districts["wave"] = np.where(districts["year"].isin(red), 'Red', districts["wave"])
districts["wave"] = np.where(districts["year"].isin(blu), 'Blue', districts["wave"])

## Bin the data and add the associated probabilities

After running multiple models on the data, it seemed that the raw probibilities (with some binning) were the best associated values. For all PVIs betwee D+5 and R+4 the probabilities speak for themselves and then for the more extreme ones are binned to remove noise

In [5]:
#split the dataframe into the smoothed bins
districts["bin"] = pd.cut(districts['metric'], 
    [0, .24, .42, .44, .45, .46, .47, .48, .49, .5,
        .51, .52, .53, .54, .57, .7, 1],
    labels=["D+26+","D+25 to D+8","D+7 to D+6","D+5","D+4","D+3","D+2","D+1",
        "EVEN","R+1","R+2","R+3","R+4","R+5 to R+7","R+8 to R+20","R+21+"])

In [6]:
def agg_probs(df):
    '''Aggregate probailites of GOP represenation by bin
    Args:
        a datdistricts with historical election results
    Returns:
        a dataframe aggregating the probabilities of GOP representation
    '''
    sort = df.groupby("bin").mean()
    sort = sort.sort_values(by="metric")
    return pd.DataFrame(sort.is_GOP).reset_index().rename(columns={"is_GOP":"prob_GOP"})

In [7]:
#aggregate probabilites of historical data
prob_gops = agg_probs(districts)
#attach associated probabilities
districts = pd.merge(districts,prob_gops)

# Compute Fairness

In [8]:
#create empty columns to fill in the state dataset
all_states["dist_loss"] = 0
all_states["porp_loss"] = 0

In [9]:
def fairness(year):
    '''A function to comupte the fairness metric for a given district
    Args: 
        a year
    Returns:
        a row in a state dataframe with a ton more columns showing fairness and displaying text
    '''
    #pull in datasets
    d = districts[districts["year"] == year]
    s = all_states[all_states["year"] == year]
    for ST in s.ST:
        st = d[d["ST"] == ST]
        st_avg = st.prob_GOP.mean()
        num_dist = len(st)
        act_num = st_avg * num_dist
        st_lean = s["metric"][s["ST"] == str(ST)]
        best_num = st_lean * num_dist
        diff_dist = (best_num-act_num)
        diff_pop = (diff_dist / num_dist)
        s.dist_loss[s["ST"] == ST] = diff_dist
        s.porp_loss[s["ST"] == ST] = diff_pop
        s["porp_text"] = np.where(s["porp_loss"] <= 0, '% Excluded, Against Dems', '% Excluded, Against GOP')
        s["porp_text"] = ((s["porp_loss"].abs().round(2))*100).fillna(0).astype(int).astype(str).str.rstrip(".0") + s["porp_text"]
        s["porp_text"] = np.where(s["porp_loss"].isna(), 'Incomplete Map', s["porp_text"])
        s["dist_text"] = np.where(s["dist_loss"] <= 0, ' Dem Districts Lost', ' GOP Districts Lost')
        s["dist_text"] = s["dist_loss"].abs().round(2).astype(str).str.rstrip(".0") + s["dist_text"]
        s["lean"] = np.where(s["porp_loss"] <= 0, 'Dems Excluded', 'GOP Excluded')
    return s

## Produce Dataset for hexmap

In [10]:
#create the 118 dataframe for major displays
state_118 = fairness(2022)
#highlight single districts and preview maps
sds_118 = ["VT", "DE", "WY", "ND", "SD", "AK"]
prev_states = ["FL","LA","NH","MO"]
state_118['State Status'] = 'Multi-District'
state_118['State Status'] = np.where(state_118["ST"].isin(sds_118), 'Single District', state_118['State Status'])
state_118['State Status'] = np.where(state_118["ST"].isin(prev_states), 'Incomplete Map', state_118['State Status'])

In [11]:
#import hexagon locations and pair them
hexes = pd.read_excel("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/raw_data/hexmap_plots.xlsx")
hexes = hexes.rename(columns={"Abbreviation":"ST"})
hexes["row"] = hexes["Row"]*-1
hexes["column"] = hexes["Column"]
prez = state_118.merge(hexes, on="ST")
#invert hexes so map is upright
#drop and rename columns
prez = prez.drop(columns=["Row","Column","year"])
prez = prez.set_axis(['ST', 'PVI', 'Share GOP','dist_loss', 'porp_loss','Population Excluded', 'Districts Lost','Lean', 'State Status','State', 'row', 'column'], axis=1, inplace=False)

In [12]:
#perform fairness on the current congress
state_117 = fairness(2020)
ref_117 = state_117.merge(hexes, on="ST")
ref_117 = ref_117.drop(columns=["Row","Column","year"])
ref_117
ref_117 = ref_117.set_axis(['ST', 'PVI', 'Share GOP','dist_loss_117', 'porp_loss_117','Districts Lost (117th)', 'Population Excluded (117th)','Lean (117th)', 'State', 'row', 'column'], axis=1, inplace=False)

In [13]:
#attach 117th to it
add_to_prez = ref_117[["ST", "dist_loss_117", "porp_loss_117", "Population Excluded (117th)", "Districts Lost (117th)", 'Lean (117th)']]
prez = prez.merge(add_to_prez)
#Create an improvement metric
prez["Change"] = np.where(prez["porp_loss"] < abs(prez["porp_loss_117"]), 'Less Partisan Than 2020', 'More Partisan Than 2020')
prez["Change"] = np.where(prez["porp_loss"] == abs(prez["porp_loss_117"]), 'No Change Since 2020', prez["Change"])
prez["Change"] = np.where(prez["State Status"] == "Incomplete Map", 'Incomplete Map', prez["Change"])
prez['Districts Lost'] = np.where(prez["State Status"] == "Incomplete Map", 'Incomplete Map', prez['Districts Lost'])
prez['Population Excluded'] = np.where(prez["State Status"] == "Incomplete Map", 'Incomplete Map', prez['Population Excluded'])
prez['Districts Lost'] = np.where(prez["State Status"] == "Incomplete Map", 'Incomplete Map', prez['Districts Lost'])
prez['porp_loss'] = np.where(prez["State Status"] == "Incomplete Map", np.nan, prez['porp_loss'])

In [14]:
#fix Error States
prez["Population Excluded (117th)"][prez["ST"] == "NV"] = "Perfect Match"
#prez["Population Excluded"][prez["State"] == "Ohio (Overturned)"] = "20% Excluded, Against Dems"
#prez["Population Excluded"][prez["ST"] == "AZ"] = "10% Excluded, Against Dems"
#prez["Population Excluded"][prez["ST"] == "DE"] = "40% Excluded, Against GOP"
prez["Population Excluded (117th)"][prez["ST"] == "DE"] = "40% Excluded, Against GOP"

In [15]:
#import data from the previous dfs
#import strings from the old sheet, with headers and the two states
Cols = "ST,PVI,Share GOP,dist_loss,porp_loss,State,row,column,Population Excluded,pe_extra,Districts Lost,State Status,porp_loss_bin,Lean,dist_loss_117,porp_loss_117,Population Excluded (117th),pe_extra_117,Districts Lost (117th),State Status (117th),porp_loss_bin_117,Lean (117th),Change"
Cols = Cols.split(",")
XOH = "OH,R+6,0.56,-2.927585787909539,-0.19517238586063593,Ohio (Overturned),0,5.5,20% Excluded, Against Dems,2.93 Dem Districts Lost,Overturned Map,-2.0,Dems Excluded,-2.3115955431306805,-0.14447472144566753,14% Excluded, Against Dems,2.31 Dem Districts Lost,Multi-District,-1,Dems Excluded,More Partisan Than 2020"
XOH = pd.DataFrame(XOH.split(",")).T
XNC = "NC,R+3,0.53,-2.2426418948429436,-0.16018870677449598,North Carolina (Overturned),0,6.5,16% Excluded, Against Dems,2.24 Dem Districts Lost,Overturned Map,-2.0,Dems Excluded,-0.7676470696329112,-0.05904977458714701,6% Excluded, Against Dems,0.77 Dem Districts Lost,Multi-District,-1,Dems Excluded,More Partisan Than 2020"
XNC = pd.DataFrame(XNC.split(",")).T
#merge the two frames and fix column names
ovs = pd.concat([XOH,XNC])
ovs.columns=Cols
#merge comma deliniated cells back together
ovs["Population Excluded"] = ovs["Population Excluded"] + "," + ovs["pe_extra"]
ovs["Population Excluded (117th)"] = ovs["Population Excluded (117th)"] + ", " + ovs["pe_extra_117"]
#drop columns no longer in the prez
ovs = ovs.drop(columns=["pe_extra","pe_extra_117","porp_loss_bin", "State Status (117th)","porp_loss_bin_117"])
#merge back into the set
pd.concat([prez,ovs])

,ST,PVI,Share GOP,dist_loss,porp_loss,Population Excluded,Districts Lost,Lean,State Status,State,row,column,dist_loss_117,porp_loss_117,Population Excluded (117th),Districts Lost (117th),Lean (117th),Change
0,AL,R+15,0.65,-1.22199,-0.17457,"17% Excluded, Against Dems",1.22 Dem Districts Lost,Dems Excluded,Multi-District,Alabama,-6,7.5,-1.26848,-0.181212,1.27 Dem Districts Lost,"18% Excluded, Against Dems",Dems Excluded,Less Partisan Than 2020
1,AK,R+9,0.59,-0.363509,-0.363509,"36% Excluded, Against Dems",0.36 Dem Districts Lost,Dems Excluded,Single District,Alaska,0,0.5,-0.363509,-0.363509,0.36 Dem Districts Lost,"36% Excluded, Against Dems",Dems Excluded,Less Partisan Than 2020
2,AZ,R+3,0.53,-0.589545,-0.065505,"7% Excluded, Against Dems",0.59 Dem Districts Lost,Dems Excluded,Multi-District,Arizona,-5,3,0.0476862,0.00529847,0.05 GOP Districts Lost,"1% Excluded, Against GOP",GOP Excluded,Less Partisan Than 2020
3,AR,R+16,0.66,-1.22053,-0.305132,"31% Excluded, Against Dems",1.22 Dem Districts Lost,Dems Excluded,Multi-District,Arkansas,-5,6,-1.07641,-0.269102,1.08 Dem Districts Lost,"27% Excluded, Against Dems",Dems Excluded,Less Partisan Than 2020
4,CA,D+14,0.36,10.3589,0.19921,"2% Excluded, Against GOP",10.36 GOP Districts Lost,GOP Excluded,Multi-District,California,-5,2,10.0952,0.190476,10.1 GOP Districts Lost,"19% Excluded, Against GOP",GOP Excluded,More Partisan Than 2020
5,CO,D+3,0.47,0.189334,0.0236667,"2% Excluded, Against GOP",0.19 GOP Districts Lost,GOP Excluded,Multi-District,Colorado,-4,3.5,0.475519,0.0679313,0.48 GOP Districts Lost,"7% Excluded, Against GOP",GOP Excluded,Less Partisan Than 2020
6,CT,D+7,0.43,1.4191,0.283819,"28% Excluded, Against GOP",1.42 GOP Districts Lost,GOP Excluded,Multi-District,Connecticut,-3,11,1.45923,0.291846,1.46 GOP Districts Lost,"28% Excluded, Against GOP",GOP Excluded,Less Partisan Than 2020
7,DE,D+6,0.44,0.395414,0.395414,"4% Excluded, Against GOP",0.4 GOP Districts Lost,GOP Excluded,Single District,Delaware,-4,9.5,0.395414,0.395414,"40% Excluded, Against GOP","4% Excluded, Against GOP",GOP Excluded,No Change Since 2020
8,FL,R+3,0.53,-2.17881,NaN,Incomplete Map,Incomplete Map,Dems Excluded,Incomplete Map,Florida,-8,8.5,-0.181692,-0.00672933,0.18 Dem Districts Lost,"1% Excluded, Against Dems",Dems Excluded,Incomplete Map
9,GA,R+3,0.53,-1.51253,-0.108038,"11% Excluded, Against Dems",1.51 Dem Districts Lost,Dems Excluded,Multi-District,Georgia,-7,8,-1.51782,-0.108415,1.52 Dem Districts Lost,"11% Excluded, Against Dems",Dems Excluded,Less Partisan Than 2020


In [16]:

#first, fix the basic zero rounding erros
prez["Population Excluded"] = np.where(prez["Population Excluded"].str[0]=="%", ("0" + prez["Population Excluded"]), prez["Population Excluded"])

#identify possible string rounding errors in the master

# limit rows to those with a single digit reported percent
low_pct = prez[prez["Population Excluded"].str[1] == "%"]

# further limit to only those with possible rounding errors
low_pct = low_pct[low_pct["Population Excluded"].str[0].astype(int) < 5]

#now eliminate all correct rows that show a low percent 
low_pct = low_pct[round(abs(low_pct["porp_loss"]),2).astype(str).str[2] != "0"]

#All of the remaining rows have the rounding error that excludes the zero

In [17]:
def fix_round_ST(ST):
    '''A function that adds a zero to a string when the rounding excluded a 0
    Args:
        A state
    Returns:
        The associated row in the master dataframe now gets its deserved zero
    
    '''
    prez["Population Excluded"][prez["ST"] == ST] = (prez["Population Excluded"].str[0] + "0" + prez["Population Excluded"].str[1:])

In [18]:
#apply the fundtion to all erroneous year-state combinations
[fix_round_ST(ST) for ST in low_pct.ST]

[None, None, None, None]

In [19]:
#Export
prez.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/2022_redraw_workbook.csv", index = False)

In [20]:
#make a quick state name key
abbrev = prez[["ST","State"]]

## Produce dataset for Slider Bar

In [21]:
master = [fairness(year) for year in all_states.year.unique()]
master = pd.concat(master)

In [22]:
master = master.merge(abbrev, on="ST")

In [23]:
#create a column that verbally represents the congress
master["congress"] = (.5*(master["year"])-893).astype(str).str.rstrip(".0")
master["Period"] = master["congress"] + "th Congress (" + master["year"].astype(str) + ")"
master = master.replace('11th Congress (2006)','110th Congress (2006)')
master["congress"] = master["congress"].astype(int)

In [24]:
#catalogue Redistricting Events
master["event"] = "No Change Since Last Election"
master["event"] = np.where(master["year"] == 2004, '2004 Election and TX redraw', master["event"])
master["event"] = np.where(master["year"] == 2008, '2008 Election', master["event"])
master["event"] = np.where(master["year"] == 2012, '2012 Election and Redistricting', master["event"])
master["event"] = np.where(master["year"] == 2016, '2016 Election and FL/NC/VA redraw', master["event"])
master["event"] = np.where(master["year"] == 2018, 'PA Redraw', master["event"])
master["event"] = np.where(master["year"] == 2020, '2020 Election', master["event"])
master["event"] = np.where(master["year"] == 2022, '2022 Redistricting', master["event"])

In [25]:
#place identifying strings on maps not yet passed
master["State"] = np.where((master["ST"].isin(prev_states)) & (master["year"]==2022), (master["State"] + " (Anticipated)"), master["State"])
#master["ST"] = np.where((master["ST"].isin(prev_states)) & (master["year"]==2022), (master["ST"] + " (Anticipated)"), master["ST"])

In [26]:
#compute aggregate loss of districts and add it to the dataframe
# create dataset for total_dist_loss_lean with an abbreviated name
tdll = master.groupby(["year","lean"]).sum().reset_index().drop(["metric","congress","porp_loss"],axis=1)
tdll["ST"] = "US"

# create dataset for total_dist_loss_year with an abbreviated name
tdly = master.groupby(["year"]).sum().reset_index().drop(["metric","congress","porp_loss"],axis=1)

#incorporate into main 
tdll = tdll.rename(columns={"dist_loss":"dist_loss_agg"})
tdly = tdly.rename(columns={"dist_loss":"dist_loss_net"})
year_lean_measures = pd.merge(tdll,tdly,on="year")
agg_dl = year_lean_measures
agg_dl["key"] = agg_dl["year"].astype(str) + agg_dl["lean"].str[0]
agg_dl = agg_dl.drop(["year","lean","ST"], axis = 1)
master["key"] = master["year"].astype(str) + master["lean"].str[0]
master = pd.merge(master,agg_dl)
master = master.drop(["key"], axis = 1)

tdll.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/total_dist_loss_lean.csv", index = False)

In [27]:

#first, fix the basic zero rounding erros
master["porp_text"] = np.where(master["porp_text"].str[0]=="%", ("0" + master["porp_text"]), master["porp_text"])

#identify possible string rounding errors in the master

# limit rows to those with a single digit reported percent
low_pct = master[master["porp_text"].str[1] == "%"]

# further limit to only those with possible rounding errors
low_pct = low_pct[low_pct["porp_text"].str[0].astype(int) < 5]

#now eliminate all correct rows that show a low percent 
low_pct = low_pct[round(abs(low_pct["porp_loss"]),2).astype(str).str[2] != "0"]

#All of the remaining rows have the rounding error that excludes the zero
#export a list of the year and state
low_pct["STyear"] = (low_pct["ST"]+low_pct["year"].astype(str))

In [28]:
def fix_round(STyear):
    '''A function that adds a zero to a string when the rounding excluded a 0
    Args:
        A year and a state
    Returns:
        The associated row in the master dataframe now gets its deserved zero
    
    '''
    ST = STyear[0:2]
    year = int(STyear[2:])
    master["porp_text"][(master["ST"] == ST) & (master["year"] == year)] = (master["porp_text"].str[0] + "0" + master["porp_text"].str[1:])

In [29]:
#apply the fundtion to all erroneous year-state combinations
[fix_round(STyear) for STyear in low_pct.STyear]
#check AL in 2004
master.head(1)

,year,ST,PVI,metric,dist_loss,porp_loss,porp_text,dist_text,lean,State,congress,Period,event,dist_loss_agg,dist_loss_net
0,2004,AL,R+8,0.58,-1.403359,-0.20048,"20% Excluded, Against Dems",1.4 Dem Districts Lost,Dems Excluded,Alabama,109,109th Congress (2004),2004 Election and TX redraw,-27.822022,-2.335246


In [30]:
master.to_csv("/Users/xavier/Desktop/DSPP/solo_projects/redistricting_project/processed/aggregated_fairness.csv", index = False)

## Spare Code

In [32]:
tdll

,year,lean,dist_loss_agg,ST
0,2004,Dems Excluded,-27.822022,US
1,2004,GOP Excluded,25.486776,US
2,2006,Dems Excluded,-27.822022,US
3,2006,GOP Excluded,25.486776,US
4,2008,Dems Excluded,-30.783795,US
5,2008,GOP Excluded,24.889357,US
6,2010,Dems Excluded,-30.783795,US
7,2010,GOP Excluded,24.889357,US
8,2012,Dems Excluded,-34.761545,US
9,2012,GOP Excluded,28.313955,US


In [ ]:
pvi_118 = districts[districts["year"] == 2022]
df = districts[districts["year"] != 2022]